In [7]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Generator model
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(28 * 28, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Generator input
latent_dim = 100
generator = build_generator(latent_dim)

# Discriminator input
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))
discriminator.trainable = False

# Combined model
gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

# Training loop
batch_size = 32
epochs = 10000
sample_interval = 1000

(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 127.5 - 1.0
X_train = np.expand_dims(X_train, axis=3)

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Create TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(X_train.shape[0]).batch(batch_size)

@tf.function
def train_step(images):
    noise = tf.random.normal((batch_size, latent_dim))
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)
        disc_loss_real = tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output)

        disc_loss_fake = tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
        disc_loss = 0.5 * (disc_loss_real + disc_loss_fake)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def generate_and_save_images(epoch):
    noise = tf.random.normal((25, latent_dim))
    generated_images = generator(noise, training=False)
    generated_images = 0.5 * generated_images + 0.5

    fig, axs = plt.subplots(5, 5)
    idx = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(generated_images[idx, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            idx += 1
    plt.savefig(f"gan_samples_epoch{epoch}.png")
    plt.close()

# Create optimizers
generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

# Training loop
for epoch in range(epochs):
    for images in dataset:
        train_step(images)

    if epoch % sample_interval == 0:
        print(f"Epoch: {epoch} | Discriminator loss: {disc_loss.numpy()} | Generator loss: {gen_loss.numpy()}")
        generate_and_save_images(epoch)


NameError: name 'disc_loss' is not defined

In [8]:
from PIL import Image

# Open the dog and pizza images
dog_image = Image.open("dog.jpg")
pizza_image = Image.open("pizza.jpg")

# Resize the images to have the same dimensions
dog_image = dog_image.resize(pizza_image.size)

# Create a new image with the same size as the pizza image
composite_image = Image.new("RGB", pizza_image.size)

# Paste the dog image on top of the pizza image
composite_image.paste(pizza_image, (0, 0))
composite_image.paste(dog_image, (0, 0), dog_image)

# Save the composite image
composite_image.save("dog_eating_pizza.jpg")


ValueError: bad transparency mask

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Generator model
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(784, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Discriminator model
def build_discriminator():
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# GAN model combining generator and discriminator
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# MNIST dataset loading and preprocessing
(x_train, _), (_, _) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32')
x_train = (x_train - 127.5) / 127.5

# Generator, discriminator, and GAN initialization
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Training loop
epochs = 100
batch_size = 128
steps_per_epoch = x_train.shape[0] // batch_size

# Compile the discriminator model
discriminator.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

for epoch in range(epochs):
    for step in range(steps_per_epoch):
        # Generate random noise and create fake images
        noise = np.random.normal(size=(batch_size, latent_dim))
        fake_images = generator.predict(noise)

        # Select a random batch of real images
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_images = x_train[idx]

        # Train the discriminator
        combined_images = np.concatenate([fake_images, real_images])
        labels = np.concatenate([np.zeros((batch_size, 1)), np.ones((batch_size, 1))])
        discriminator_loss = discriminator.train_on_batch(combined_images, labels)

        # Train the generator (via GAN)
        noise = np.random.normal(size=(batch_size, latent_dim))
        labels = np.ones((batch_size, 1))
        gan_loss = gan.train_on_batch(noise, labels)

    # Print training progress
    print(f'Epoch {epoch+1}/{epochs} | Discriminator Loss: {discriminator_loss} | GAN Loss: {gan_loss}')

# Generate and display some fake images
noise = np.random.normal(size=(10, latent_dim))
generated_images = generator.predict(noise) * 0.5 + 0.5

for i in range(10):
    image = generated_images[i, :, :, 0]
    image = (image * 255).astype(np.uint8)
    Image.fromarray(image).show()


4/4 [==============================] - 0s 7ms/step


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [2]:
import pygame
import random
from PIL import Image

# Initialize Pygame
pygame.init()

image_width = 500
image_height = 500

# Set the screen dimensions
screen_width = max(800, image_width)  # Replace 800 with the desired minimum screen width
screen_height = max(600, image_height)  # Replace 600 with the desired minimum screen height
screen = pygame.display.set_mode((screen_width, screen_height))

# Load and resize the image
image_path = "pizza.jpg"  # Replace with the actual image file path
image = Image.open(image_path)
if image.width > screen_width or image.height > screen_height:
    image.thumbnail((screen_width, screen_height))
image = pygame.image.fromstring(image.tobytes(), image.size, image.mode)



# Generate random initial positions for the image
x = random.randint(0, screen_width - image_width) if screen_width > image_width else 0
y = random.randint(0, screen_height - image_height) if screen_height > image_height else 0

# Generate random velocities for the image
vx = random.uniform(-1, 1)  # Horizontal velocity
vy = random.uniform(-1, 1)  # Vertical velocity

# Main game loop
running = True
while running:
    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update the image position
    x += vx
    y += vy

    # Check if the image is going off the screen
    if x < 0 or x + image_width > screen_width:
        vx *= -1  # Reverse horizontal velocity
    if y < 0 or y + image_height > screen_height:
        vy *= -1  # Reverse vertical velocity

    # Draw the image on the screen
    screen.fill((0, 0, 0))  # Clear the screen
    screen.blit(image, (x, y))  # Draw the image at (x, y)
    pygame.display.flip()  # Update the display

# Quit the game
pygame.quit()
